In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib as mpl

import math 

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

## Setup

In [3]:
n = 2000
d = 0.3;
σ = 8.0

[a1,b1],[a2,b2] = support_gen(n,d,σ)

λmin_ = a1-.1
λmax_ = b2+.1
a = (b1+a2)/2

In [4]:
np.random.seed(0)
A = mat_samp_full(n,d,σ)@np.eye(n)
Λ = np.linalg.eigvalsh(A)
A = np.diag(Λ)

λmin = np.min(Λ)
λmax = np.max(Λ)

b = np.random.randn(n)
b /= np.linalg.norm(b)

In [5]:
w = a

# norm to measure function approximation and linear system errors
B = (Λ-w)**2

kappa_w = np.max(np.abs(Λ-w)) / np.min(np.abs(Λ-w))

In [6]:
K = 201
Q,(a_,b_) = lanczos_reorth(A,b,K,reorth=K)

In [7]:
plot_tol = 1e-14

In [8]:
T = np.zeros((3,K))
T[0,1:] = b_[:-1]
T[1] = a_-w
T[2,:-1] = b_[:-1]

e1 = np.zeros(K+1)
e1[0] = 1

In [9]:
err_CG = np.full(K,np.nan)

for k in range(0,K):
    try:
        e = (1/(Λ-w))*b - np.linalg.norm(b)*Q[:,:k+1]@sp.linalg.solve_banded((1,1),T[:,:k+1],e1[:k+1])
        err_CG[k] = np.sqrt( e.T*B@e )
    except: pass
    
    if err_CG[k] < plot_tol:
        break

In [10]:
res_CG = np.full(K,np.nan)

for k in range(0,K):
    try:
        r = b - np.linalg.norm(b)*(Λ-w)*(Q[:,:k+1]@sp.linalg.solve_banded((1,1),T[:,:k+1],e1[:k+1]))
        res_CG[k] = np.sqrt( r@r )
    except: pass
    
    if res_CG[k] < plot_tol:
        break

## $f(x) = \operatorname{step}(x-a)$, $\Gamma =$ double circles

In [11]:
def f(x):
    return (np.abs(np.real(x))>a)/x

fAb = f(Λ)*b

In [12]:
err_lanczos = np.full(K,np.nan)

for k in range(0,K):
    try:
        e = fAb - lanczos_FA(f,Q,a_,b_,k,normb=np.linalg.norm(b))
        err_lanczos[k] = np.sqrt( e.T*B@e )
    except: pass
    
    if err_lanczos[k] < plot_tol:
        break

In [13]:
a_priori_bound = np.full(K,np.inf)
a_posteriori_bound = np.full(K,np.inf)

def Gamma1(t):
    radius = λmax-w

    z = radius*(np.exp(1j*t))+λmax
    dz = radius*1j*np.exp(1j*t)
    
    return z,dz

def Gamma2(t):
    radius = w-λmin

    z = radius*(np.exp(1j*t))+λmin
    dz = radius*1j*np.exp(1j*t)
    
    return z,dz


for k in range(0,K):
    a_priori_bound[k-1] = 2*(get_a_priori_bound(f,Gamma1,[0,np.pi],k,w,λmin_,λmax_)[0]+\
                             get_a_priori_bound(f,Gamma2,[0,np.pi],k,w,λmin_,λmax_)[0])
    a_posteriori_bound[k-1] = 2*(get_a_posteriori_bound(f,Gamma1,[0,np.pi],a_[:k],b_[:k-1],w,λmin_,λmax_)[0]+\
                                 get_a_posteriori_bound(f,Gamma2,[0,np.pi],a_[:k],b_[:k-1],w,λmin_,λmax_)[0])

In [14]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))
ax = fig.add_axes([left, bottom, width, height])

ks = np.arange(K)

ax.plot(ks,((λmax-a)/a*err_CG),\
        **line_styles['l4'])

ax.plot(ks,(a_priori_bound*err_CG),\
        **line_styles['l3'])
ax.plot(ks,(a_posteriori_bound*err_CG),\
        **line_styles['l2'])

ax.plot(ks,err_lanczos,\
        **line_styles['l1'])

ax.set_yscale('log')

plt.grid(True,linestyle=':',linewidth=.5)

#plt.ylim(1e-14,5e-1)

plt.xlabel('number of matvecs: $k$')

plt.savefig('imgs/ch7_pcr.pdf')
#plt.close()